# Seminární úloha - korelace

In [3]:
import numpy as np
from scipy.stats import norm

# --- Vstupní data ---
a = np.array([30, 31, 39, 40, 41, 42, 39, 45, 36, 46])
b = np.array([10.1, 9.5, 12.1, 12.5, 13.5, 12.4, 11.4, 12.6, 8.8, 13.0])
c = np.array([9.9, 9.5, 9.2, 9.0, 9.1, 8.9, 9.3, 8.8, 10.2, 8.7])

N = len(a) # Počet měření

# --- Průměry a směrodatné odchylky ---
# Používáme ddof=1 pro výběrovou směrodatnou odchylku (s)
mean_a = np.mean(a)
std_a = np.std(a, ddof=1)

mean_b = np.mean(b)
std_b = np.std(b, ddof=1)

mean_c = np.mean(c)
std_c = np.std(c, ddof=1)

# --- Průměry součinů (pro kontrolu, i když korelaci počítáme přímo) ---
mean_ab = np.mean(a * b)
mean_bc = np.mean(b * c)
mean_ac = np.mean(a * c)

# --- Korelační koeficienty ---
# Korelační matice: np.corrcoef(x, y)[0, 1] vrací r_xy
r_ab = np.corrcoef(a, b)[0, 1]
r_bc = np.corrcoef(b, c)[0, 1]
r_ac = np.corrcoef(a, c)[0, 1]

# Společná standardní odchylka
sigma_F = 1 / np.sqrt(N - 3) 
print(f"Standardní odchylka transformace (σ): {sigma_F:.4f}")
alpha = 0.05

def test_korelace(r, name):
    # 1. Fisherova transformace
    F_r = 0.5 * np.log((1 + r) / (1 - r))
    
    # 2. Testovací statistika z
    z = F_r / sigma_F
    
    # 3. p-hodnota (dvoustranný test)
    p_value = 2 * (1 - norm.cdf(np.abs(z)))
    
    # 4. Závěr
    zavěr = "Signifikantní (Zamítáme H₀)" if p_value < alpha else "Není signifikantní (Nezamítáme H₀)"
    
    print(f"\nPro pár {name} (r={r:.4f}):")
    print(f"  F(r): {F_r:.4f}")
    print(f"  Testová statistika z: {z:.4f}")
    print(f"  p-hodnota: {p_value:.4f}")
    print(f"  Závěr (α={alpha}): {zavěr}")

test_korelace(r_ab, 'a, b')
test_korelace(r_bc, 'b, c')
test_korelace(r_ac, 'a, c')

Standardní odchylka transformace (σ): 0.3780

Pro pár a, b (r=0.8109):
  F(r): 1.1297
  Testová statistika z: 2.9890
  p-hodnota: 0.0028
  Závěr (α=0.05): Signifikantní (Zamítáme H₀)

Pro pár b, c (r=-0.8956):
  F(r): -1.4493
  Testová statistika z: -3.8346
  p-hodnota: 0.0001
  Závěr (α=0.05): Signifikantní (Zamítáme H₀)

Pro pár a, c (r=-0.8167):
  F(r): -1.1469
  Testová statistika z: -3.0345
  p-hodnota: 0.0024
  Závěr (α=0.05): Signifikantní (Zamítáme H₀)


In [4]:
print("\n--- Odhad hodnoty a chyby veličiny y = 3ab/c² ---")

# --- 3.A: Odhad očekávané hodnoty (y_hat) ---
y_hat = (3 * mean_a * mean_b) / (mean_c**2)
print(f"Odhad očekávané hodnoty ŷ: {y_hat:.4f}")

# --- 3.B: Odhad chyby (Propagace chyby) ---

# 1. Parciální derivace v průměrech
dy_da = (3 * mean_b) / (mean_c**2)
dy_db = (3 * mean_a) / (mean_c**2)
dy_dc = (-6 * mean_a * mean_b) / (mean_c**3)

# 2. Vypočet variance (s_y^2)

# Vlastní členy (diagonální)
var_y_diag = (dy_da**2 * std_a**2) + (dy_db**2 * std_b**2) + (dy_dc**2 * std_c**2)

# Kovarianční členy (off-diagonální)
cov_ab = 2 * dy_da * dy_db * (r_ab * std_a * std_b)
cov_ac = 2 * dy_da * dy_dc * (r_ac * std_a * std_c)
cov_bc = 2 * dy_db * dy_dc * (r_bc * std_b * std_c)

# Celková variance a chyba
var_y = var_y_diag + cov_ab + cov_ac + cov_bc
std_y = np.sqrt(var_y)

print(f"Parc. derivace dy/da: {dy_da:.4f}")
print(f"Parc. derivace dy/db: {dy_db:.4f}")
print(f"Parc. derivace dy/dc: {dy_dc:.4f}")
print(f"Celková variance s_y²: {var_y:.4f}")
print(f"Odhad chyby (sm. odchylka s_y): {std_y:.4f}")

print("\n--- Finální výsledek ---")
print(f"Veličina y je odhadnuta jako: y = {y_hat:.2f} ± {std_y:.2f}")


--- Odhad hodnoty a chyby veličiny y = 3ab/c² ---
Odhad očekávané hodnoty ŷ: 15.7737
Parc. derivace dy/da: 0.4055
Parc. derivace dy/db: 1.3610
Parc. derivace dy/dc: -3.4068
Celková variance s_y²: 31.7570
Odhad chyby (sm. odchylka s_y): 5.6353

--- Finální výsledek ---
Veličina y je odhadnuta jako: y = 15.77 ± 5.64
